# Evaluation Report

In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from src.dataset.dataset_coco import CocoDataset
from src.architectures.detector_maskrcnn import MaskRCNNDetector
from torch.utils.data import DataLoader
from sklearn.metrics import (
    confusion_matrix,
    ConfusionMatrixDisplay,
    PrecisionRecallDisplay,
    recall_score,
    precision_score,
    accuracy_score,
    f1_score,
)

Name your experiment

In [ ]:
EXPERIMENT_NAME = "ai-palatal-rugae-exp01"

Set dataset and model variables

In [ ]:
# ~~ Dataset

testset_annotations_path = f""
testset_images_path = f""
dataset = CocoDataset(testset_images_path, testset_annotations_path)
dataloader = DataLoader(dataset, batch_size=1, shuffle=False)

In [ ]:
# ~~ Model
model_path = f"{EXPERIMENT_NAME}/checkpoint.pth"
model = MaskRCNNDetector(model_path)
model.load()
model.to(torch.device("cuda:0"))    # Load model on GPU

Run model over the input images.

In [4]:
preds = []
truths = []

model.eval()

with torch.no_grad():
    for batch in dataloader:
        x_pred, y_true = batch
        x_pred = x_pred.to("cuda:0")

        y_pred = model(x_pred)

        preds.append(y_pred.cpu().detach().numpy()[0])
        truths.append(y_true.cpu().detach().numpy()[0])

Compute evaluation metrics.

In [ ]:
PrecisionRecallDisplay.from_predictions(truths, preds)
plt.show()

In [ ]:
truths_classes = np.array(truths, dtype=np.uint8)
preds_classes = np.array(np.round(preds), dtype=np.uint8)

conf_mat = confusion_matrix(truths_classes, preds_classes, labels=[0, 1])
ConfusionMatrixDisplay(conf_mat, display_labels=["rugae_01", "rugae_02"]).plot()

plt.show()

In [ ]:
precision = precision_score(truths_classes, preds_classes)
recall = recall_score(truths_classes, preds_classes)
accuracy = accuracy_score(truths_classes, preds_classes)
f1score = f1_score(truths_classes, preds_classes)

print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1score}")

Visualize some predictions

In [ ]:
random_index = np.random.randint(0, len(dataset))
image, ground_truth = dataset[random_index]
image = image.unsqueeze(0)
image = image.to("cuda:0")

y_pred = model(image)
y_pred = y_pred.cpu().detach().numpy()[0]
y_pred = np.round(y_pred)

image = image.cpu().detach().numpy()[0]
image = image.transpose(1, 2, 0)
image = image.astype(int)

print(f"Prediction: {y_pred} / Ground Truth: {ground_truth}")

plt.imshow(image)
plt.show()